# Real Time Sonar App



In [1]:
# Import functions and libraries
from __future__ import division
import numpy as np
from scipy import signal
from numpy import *

from rtsonar import rtsonar

Loading BokehJS ...

### Task III: Play with different parameters with the real-time sonar!

The function `rtsonar()` in `rtsonar.py` provides a wrapper to create a real-time sonar plot using user-defined processing functions. Copy and paste your functions genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time() to the code cell below. Scroll down to the bottom and you should be able to run `rtsonar()`!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can buy the time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your laptop, moving yourself next to the computer, using a book as a reflecting surface. It would be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 

In [2]:
# Copy and paste your 5 functions here:
# genPulseTrain()
def genPulseTrain(pulse, Nrep, Nseg):
    # Function generates a pulse train from a pulse. 
    # Inputs:
    #     pulse  = the pulse generated by genChirpPulse
    #     Nrep   = number of pulse repetitions
    #     Nseg   = length of pulse segment >= len(pulse)
    # Output:
    #     ptrain = pulse train
    
    seg = zeros(Nseg)
    seg[:len(pulse)] = pulse
    ptrain = tile(seg, Nrep)
    
    return ptrain
# genChirpPulse()
def genChirpPulse(Npulse, f0, f1, fs):
    # Function generates an analytic function of a chirp pulse
    # Inputs:
    #     Npulse = pulse length in samples
    #     f0     = starting frequency of chirp
    #     f1     = end frequency of chirp
    #     fs     = sampling frequency
    # Output:
    #     pulse  = chirp pulse
    
    T = Npulse/fs
    t = np.r_[0.0:Npulse]/fs
    k = (f1-f0)/T
    pulse = exp(1j*2*pi*(f0 + k/2*t)*t)
    
    return pulse
# crossCorr()
def crossCorr(rcv, pulse_a):
    # Funtion generates cross-correlation between rcv and pulse_a.
    # Inputs:
    #     rcv     = received signal
    #     pulse_a = analytic pulse
    # Output:
    #     Xrcv    = cross-correlation between rcv and pulse_a
    
    Xrcv = signal.convolve(rcv, pulse_a[::-1])
    
    return Xrcv
# findDelay()
def findDelay(Xrcv, Nseg):
    # Function finds the first pulse index.
    # Inputs:  
    #     Xrcv = the received matched filtered signal
    #     Nseg = length of a segment
    # Output:
    #     ind  = index of the beginning of the first pulse
    
    ind = argmax(Xrcv[0:Nseg])
    
    return ind
# dist2time()
def dist2time(dist, temperature=21.0):
    # Function converts distance in cm to time in seconds.
    # Inputs:
    #     dist        = distance to object in cm
    #     temperature = in Celcius
    # Output:
    #     t           = time in seconds between transmitted pulse and echo
    
    #Your code here
    t= dist/331.5/np.sqrt(1+temperature/273.15)/100*2
    
    return t

In [3]:
# Run this for Real-time Sonar
# Change the parameters!
fs = 44100 # Sampling frequency
f0 = 10000 # Chirp initial frequency
f1 = 15000 # Chirp ending frequency

Npulse = 300 # Length of Chirp Pulse
Nrep = 48 # Number of repetition in a pulse train (determines vertical length of plot )
Nseg = 2048*8 # Number of samples between pulses (determines maximum time-of-arrival)
Nplot = 770 # Number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist = 300 # Maximum distance in cm
temperature = 20 # Temperature     

functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) #join the functions together

stop_flag = rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions )

In [4]:
# Run this to stop the sonar
stop_flag.set()